# MultiProcessing, Strong Scaling

François-David Collin (CNRS, IMAG, Paul-Valéry Montpellier 3
University)  
Ghislain Durif (CNRS, LBMC)  
Monday, August 26, 2024

## ⚠️⚠️⚠️⚠️ Attention ⚠️⚠️⚠️⚠️

Under *Windows*, with python the multiprocessing module
`multiprocessing` works in a normal script but **not in notebooks**.

If you absolutely must use Windows,
use[WSL](https://docs.microsoft.com/fr-fr/windows/wsl/)

# Strong Scaling

## Prerequisites

For this TP, set the number of **physical** cores available (8 on the
cluster nodes), not the number of logical cores.

In [2]:
ncores = 8 # 8 on the cluster nodes

In [3]:
import math

# Introduction

## Basic functions

Make a function `is_prime` that tests whether an integer $n$ strictly
greater than 2 is prime or not.

Hint: First check that it is not even, then list all odd numbers from 3
to $\sqrt{n}$ (upper rounding) and test whether they are factors.

In [4]:
def is_prime(n):
    if n % 2 == 0:
        return False
    for i in range(3, int(math.sqrt(n)) + 1, 2):
        if n % i == 0:
            return False
    return True

Make a function `total_primes` that counts the number of primes in a
list.

In [5]:
def total_primes(l):
    n = 0
    for i in l:
        if (i > 0) & (i <= 2):
            n=n+1
        elif is_prime(i):
            n=n+1                
    return n

Calculate the number of primes from 1 to $N=100 000$ with this function

In [6]:
N=100000

In [7]:
total_primes(range(1,N+1))

9593

## Time measurement

Use `%timeit` to measure the average time taken to count the number of
primes up to $N=100000$. (note: by default, `timeit` repeats the
calculation $7times{}10$ to obtain a reliable average. Please refer to
the
[magic](https://ipython.readthedocs.io/en/stable/interactive/magics.html)
and [timeit](https://docs.python.org/3.9/library/timeit.html) docs).

Store measurements using the -o option in timeit

In [8]:
orig_time = %timeit -o total_primes(range(1,N+1))

71.7 ms ± 205 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)

# First steps

Our first attempt at multiprocessing will involve partitioning the count
into 2. We’ll run two processes in parallel on $\{1,...,N/2\}$ and
$\{N/2+1,...,N\}$.

Complete the following code
([source](https://notebook.community/izapolsk/integration_tests/notebooks/MultiProcessing)).

Check the result and the performance gain with `%timeit`.

In [9]:
from multiprocessing.pool import Pool

def split_total(N):
    with Pool(2) as pool:
        return sum(pool.map(total_primes, ...))

In [10]:
from multiprocessing.pool import Pool

def split_total(N):
    with Pool(2) as pool:
        return sum(pool.map(total_primes, [range(1,int(N/2)), range(int(N/2)+1,N+1)]))

In [11]:
split_total(N)

9593

In [12]:
split_time = %timeit -o split_total(N)

51.8 ms ± 79.5 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)

In [13]:
print("Gain with split : {:.1f}".format(orig_time.average/split_time.average))

Gain with split : 1.4

# Generalization

Generalize the function with partitioning into *n* tasks instead of just
2. We’ll use a generalized `multi_process_list` function, which takes as
arguments : -
f`the main computation function, which takes an integer list as argument - n`
the number of partitions (here, one partition = task) -
par_f`a function which takes as argument a list and a number of partitions to be performed, and returns the list of partitions in this list - l`
the list to be partitioned

In [14]:
def multi_process_list(f,n,par_f,l):
    with Pool(ncores) as pool:
        return sum(pool.map(...)

First, we write the `naive_par` partitioning function.

In [15]:
def naive_par(lst,n):
    return ...

We’ll use the `chunks` function, which partitions a list into chunks of
fixed size (except for the last one).

We’ll test the gain obtained with 8 tasks/partitions.

In [16]:
def chunks(lst, m):
    """Yield successive m-sized chunks from lst."""
    for i in range(0, len(lst), m):
        yield lst[i:i + m]

Vérifier le fonctionnement de `naive_par`

In [17]:
def naive_par(lst,n):
    return chunks(lst,int(len(lst)/n))

In [18]:
list(naive_par(range(1,100001),4))

[range(1, 25001),
 range(25001, 50001),
 range(50001, 75001),
 range(75001, 100001)]

In [19]:
def multi_process_list(f,n,par_f,l):
    with Pool(n) as pool:
        return sum(pool.map(f,par_f(l,n)))

In [20]:
multi_process_list(total_primes,ncores,naive_par,range(1,N+1))

9593

In [21]:
multi_time = %timeit -o multi_process_list(total_primes,ncores,naive_par,range(1,N+1))

44.3 ms ± 1.13 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

In [22]:
print("Gain avec multi : {:.1f}".format(orig_time.average/multi_time.average))

Gain avec multi : 1.6

Repeat all calculations and payoff comparisons with $N=5000000$. To
avoid long calculation times, we’ll restrict ourselves to a single
iteration (option `-r 1 -n 1` in `%timeit`).

In [23]:
N = 5000000

In [24]:
orig_time = %timeit -r 1 -n 1 -o total_primes(range(1,N+1))
split_time = %timeit -r 1 -n 1 -o split_total(N)
multi_time = %timeit -r 1 -n 1 -o multi_process_list(total_primes,ncores,naive_par,range(1,N+1))

print("Gain with split : {:.1f}".format(orig_time.average/split_time.average))
print("Gain with multi : {:.1f}".format(orig_time.average/multi_time.average))

18.5 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
11.2 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
3.11 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
Gain with split : 1.7
Gain with multi : 5.9

# Optional refinement

How much time is spent on each task? Use the following function to get
an idea. What do you observe?

In [25]:
def timed_total_primes(l):
    %timeit -r 1 -n 1 total_primes(l)
    return 0

In [26]:
multi_process_list(timed_total_primes,ncores,naive_par,range(1,N+1))

921 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
1.6 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
1.96 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
2.23 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
2.48 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
2.7 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
2.89 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
3.23 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)

0

How can we solve this problem?

Find a simple solution that requires only one line of code. Check the
execution times of individual tasks.

Compare again with $N = 10000000$ (which will take about 1 minute
sequentially).

In [27]:
import random

N = 10000000

shuffled = random.sample(range(1,N+1),N)

In [28]:
multi_process_list(total_primes,ncores,naive_par,shuffled)

664580

In [29]:
temps_shuffled = %timeit -r 1 -n 1 -o multi_process_list(total_primes,ncores,naive_par,shuffled)

7.68 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)

In [30]:
multi_process_list(timed_total_primes,ncores,naive_par,shuffled)

5.98 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
5.98 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
5.98 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
5.96 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
5.99 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
5.95 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
5.94 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
6.16 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)

0

In [31]:
shuffled = random.sample(range(1,N+1),N)
orig_time = %timeit -r 1 -n 1 -o total_primes(range(1,N+1))
multi_time = %timeit -r 1 -n 1 -o multi_process_list(total_primes,ncores,naive_par,range(1,N+1))
shuffled_time = %timeit -r 1 -n 1 -o multi_process_list(total_primes,ncores,naive_par,shuffled)

print("Gain with multi : {:.1f}".format(orig_time.average/multi_time.average))
print("Gain with shuffled : {:.1f}".format(orig_time.average/shuffled_time.average))

48 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
8.31 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
7.67 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
Gain with multi : 5.8
Gain with shuffled : 6.3

# Recreational interlude

If you have a CPU with SMT (Hyperthreading), redo the measurements with
`ncores` equal to the number of logic cores, and explain the results.

In [32]:
ncores = 8 # On a machine with 4 physical cores/8 logical cores

shuffled = random.sample(range(1,N+1),N)
orig_time = %timeit -r 1 -n 1 -o total_primes(range(1,N+1))
multi_time = %timeit -r 1 -n 1 -o multi_process_list(total_primes,ncores,naive_par,range(1,N+1))
shuffled_time = %timeit -r 1 -n 1 -o multi_process_list(total_primes,ncores,naive_par,shuffled)

print("Gain with multi : {:.1f}".format(orig_time.average/multi_time.average))
print("Gain with shuffled : {:.1f}".format(orig_time.average/shuffled_time.average))

This has been run on a machine with 4 physical cores and 8 logical
cores, [Intel Xeon CPU E5-1620
v4](https://www.intel.com/content/www/us/en/products/sku/92991/intel-xeon-processor-e51620-v4-10m-cache-3-50-ghz/specifications.html).

The results show that the gain is less than with 4 physical cores. This
is because the logical cores share the same physical core, and the
overhead of sharing the same physical core is greater than the gain from
parallelization.

    37.4 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
    12.1 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
    12.9 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
    Gain with multi : 3.1
    Gain with shuffled : 2.9